# Einfluss von Tiefe und Breite eines MLPs


In [ ]:
# Importiere Zeugs und lade Utilities
import torch
import matplotlib.pyplot as plt
from tqdm.auto import trange
! wget -nc https://raw.githubusercontent.com/inb-luebeck/CS4230-S/main/utils/utils_mlp.py
from utils_mlp import MLP, generate_plot

# Generiere Datensatz (Two Spiral Problem)
n_data = 500
base = torch.linspace(-1,1, steps=n_data)*torch.pi*2
radius = torch.linspace(0.5,3, steps=n_data)
data = torch.stack([torch.sin(base)*radius, torch.cos(base)*radius]).T
data = torch.row_stack([data[::2], data[1::2]])
data += 0.13*torch.randn([n_data, 2])
m1 = torch.tensor([0,-1])
m2 = torch.tensor([-0.5,0])
data[::2] *= -1 
label = torch.tensor(int(n_data/2)*[0,1])

# Plotte Daten
plt.figure()
plt.plot(data[label==0, 0], data[label==0, 1], "b.")
plt.plot(data[label==1, 0], data[label==1, 1], "r.")
plt.axis("equal")
plt.show()

In [ ]:
# --- Hier können die Hyperparameter angepasst werden ---
n_epochs = 1000
innerdim = 2
n_hidden = 1
learningrate = 1e-3
# -------------------------------------------------------

# Initialisiere Model, Optimierer und Lossfunktion
model = MLP(innerdim=innerdim, n_hidden=n_hidden)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learningrate)
losslist = torch.zeros(n_epochs)

# Training
for epoch in trange(n_epochs):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, label[:,None].float())
    loss.backward()
    optimizer.step()
    losslist[epoch] = loss.item()

img_plot, data_plot = generate_plot(model, data)

# Plotte Ergebnisse
plt.figure(figsize=[13,5])
plt.subplot(1,2,1)
plt.plot(losslist)
plt.subplot(1,2,2)
plt.imshow(img_plot.detach().numpy())
plt.plot(data_plot[label==0, 0], data_plot[label==0, 1], "b.")
plt.plot(data_plot[label==1, 0], data_plot[label==1, 1], "r.")
plt.show()
# Hinweis: Die zwei Spiralen scheinen andersrum zu drehen als beim Initialisieren 
#     oben! Das liegt allerdings nur an der Darstellung im Bild. Die Datenpunkte
#     sind trotzdem dieselben.